In [2]:
%pip install -U nlpaf


Requirement already up-to-date: nlpaf in /home/elba_ro/repos/github/conf22-style-transfer/iesta_venv/lib/python3.8/site-packages (2.12.20)
Note: you may need to restart the kernel to use updated packages.


In [1]:
!nvidia-smi -q

/bin/bash: nvidia-smi: command not found


In [ ]:
"""
%pip install python-dotenv
%pip install  -e ../
%pip install codecarbon
%pip install optuna
%pip install -U nlpaf
%pip install GPUtil
%pip install comet_ml
%pip install --upgrade accelerate
!nvidia-smi
"""
# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key
global experiment
experiment = Experiment(
    api_key="ZGtUiFA68BqsXMlFfogUwnIfD",
    project_name="iesta",
    workspace="roxanneelbaff",
    experiment_key = "liberalclassifier")
experiment.set_name("liberal_evaluator_25may_epoch50")

COMET_API_KEY = "ZGtUiFA68BqsXMlFfogUwnIfD"

In [ ]:

import GPUtil
import torch
torch.cuda.empty_cache()
GPUtil.showUtilization()

: 

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

os.getenv('max_split_size_mb')

In [ ]:
from iesta.machine_learning.dataloader import IESTAData, LABELS
from iesta.machine_learning.huggingface_loader import IESTAHuggingFace
from nlpaf.transformers.text_classification import TextClassification


In [ ]:

import torch
use_cuda = torch.cuda.is_available()
use_cuda

if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)
    

In [ ]:
liberal_data_obj = IESTAData(ideology="liberal", keep_labels = LABELS.EFF_INEFF)
liberal_hf = IESTAHuggingFace(liberal_data_obj)
#liberal_w_effect = liberal_hf.upload_w_labels(is_for_style_classifier=False, force_reload=True)
#liberal_w_effect_for_sc = liberal_hf.upload_w_labels(is_for_style_classifier=True, force_reload=True)


In [ ]:
liberal_sc_dataset = liberal_hf.get_dataset_name(is_for_style_classifier=True)
liberal_sc_dataset

In [ ]:
liberal_sc_trainer = TextClassification(
    liberal_sc_dataset,
    id2label=  IESTAHuggingFace._ID2LABEL_,
    label2id= IESTAHuggingFace._LABEL2ID_,
    training_key="training",
    eval_key="validation",
    text_col="text",
    undersample=True,
    pretrained_model_name="distilbert-base-uncased",#"microsoft/deberta-base",
    metric="f1",
    averaged_metric="macro",
    model_org="notaphoenix",  # only if you want to upload your model to hf
    output_dir="liberal_effective_arg_classifier",
    learning_rate= 5e-6,
    per_device_train_batch_size=4,
    per_device_eval_batch_size= 4,
    num_train_epochs= 50,
    weight_decay= 0.01,
    evaluation_strategy= "epoch",
    save_strategy= "epoch",
    load_best_model_at_end= True,
    push_to_hub= True,
    hub_private_repo= True,
    report_to="comet_ml"  # comet_ml
    )
 

#best hyperparemeters for small data with transformers for text classification

In [ ]:
#! nvcc -V

In [ ]:
liberal_sc_trainer.train()

In [ ]:
"""
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-8, 5e-5, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16 ]),
    }
hpsearch = liberal_sc_trainer.search_hyperparameters( n_trials= 5,
                               run_on_subset= False,
                               direction="maximize",
                               load_best_params=True,
                               hp_space_func=optuna_hp_space
                               )
"""

In [ ]:
experiment.end()